# Read Simulation Data
With the following code, we have developed a method for reading and transforming simulation data from JaamSim. The method consists of two main parts. The first is the `get_WIP` function, which takes data from an ExpressionLogger, and calculates the average estimated throughput, actual throughput, and Work-in-Process (WIP). The second method processes the `RunOutputList` data, transforming it into a more readable dataframe, and also printing out the LaTeX code for the table.

In [1]:
import pandas as pd

#Set full display of columns
pd.set_option('display.max_columns', None)

# Set full width of the output
pd.set_option('display.width', 1000)

# Set full display of rows
pd.set_option('display.max_rows', None)


In [2]:
def get_WIP(file_name, scenario):
    # Open the file, skip first 13 rows and the first 8000*(scenario-1) rows
    # This is because the file has 8000 rows per scenario
    with open(file_name, 'r') as f:
        lines = f.readlines()[12+8000*(scenario-1):12+8000*scenario]

    # Loop through the lines in the file, due to the format of the file
    # For every 8th row, we must skip the first 4 rows, and keep the next 4
   
    lines_time = []
    lines_e = []
    lines_a = []
    lines_wip = []

    i = 0
    for line in lines:
        if i % 8 != 0 and i % 8 != 1 and i % 8 != 2 and i % 8 != 3:
            lines_time.append(line.strip().split("\t")[0])
            lines_e.append(line.strip().split("\t")[1])
            lines_a.append(line.strip().split("\t")[2])
            lines_wip.append(line.strip().split("\t")[3])
        i += 1

    # Create a dataframe with the data
    df = pd.DataFrame(list(zip(lines_time, lines_e, lines_a, lines_wip)), columns =['Time', 'Actual', 'Estimated', 'WIP'])

    # Convert the columns to numeric
    df['Actual'] = pd.to_numeric(df['Actual'])
    df['Estimated'] = pd.to_numeric(df['Estimated'])
    df['WIP'] = pd.to_numeric(df['WIP'])

    # Calculate the mean of the WIP
    mean = df.groupby('Time').mean()
    
    # Convert from minutes to hours
    mapper = {
        "1020.0" : "17:00",
        "1080.0" : "18:00",
        "1140.0" : "19:00",
        "1200.0" : "20:00"
    }
    
    mean = mean.rename(index=mapper)

    return mean


In [3]:
def get_CT_and_Queue(file_name, scenario):
    # The order of the measures in the file
    measures = ['CT_total', 'SD_total', 'CT_FU', 'SD_FU', 'CT_R', 'SD_R', 'CT_N', 'SD_N', 'Queue_Working', 'SQ_Working', 'Queue_Wating_Room', 'SQ_Wating_Room', 'Queue_frontdesk', 'SQ_frontdesk', 'Queue_total_l', 'SQ_total_l', "CT_Working", "SD_Working", 'Queue_total_c', 'SQ_total_c', 'Queue_FU', 'SQ_FU', 'Queue_R', 'SQ_R', 'Queue_N', 'SQ_N']
    # Read the file and choose the corresponsing scenario line
    with open(file_name, "r") as file:
        lines = file.readlines()[scenario:]

    print("Read the file")
    line = lines[0].split("\t")[1:]

    # Group two and two values (value and ST. deviation) and create a dictionary with the measures as keys and the values as values
    data_dict = {}
    for i in range(0, len(measures) * 2, 2):
        measure = measures[i // 2]
        value, conf = line[i], line[i+1]
        if measure.startswith('SD'):
            # Add SD as a new column to the corresponding CT
            ct_key = measure.replace('SD', 'CT')
            if ct_key in data_dict:
                data_dict[ct_key]['SD'] = value
        elif measure.startswith('SQ'):
            # Add SD as a new column to the corresponding Queue
            queue_key = measure.replace('SQ', 'Queue')
            if queue_key in data_dict:
                data_dict[queue_key]['SD'] = value
        else:
            # Create new entry for CT measures or others
            data_dict[measure] = {'Mean': value, 'SD': None, 'Confidence': conf}  # Initialize SD and Confidence as None

    # Create a dataframe from the dictionary
    df = pd.DataFrame.from_dict(data_dict, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Measure'}, inplace=True)

    # Convert columns to numeric and calculate confidence intervals
    df['Mean'] = pd.to_numeric(df['Mean'])
    df['SD'] = pd.to_numeric(df['SD'])
    df['Confidence'] = pd.to_numeric(df['Confidence'])
    df['Confidence_interval_1'] = df['Mean'] - df['Confidence']
    df['Confidence_interval_2'] = df['Mean'] + df['Confidence']
    df  = df.round(3)

    # Create a new column 'Confidence Interval'
    df['Confidence Interval'] = "[" + df['Confidence_interval_1'].astype(str) + ", " + df['Confidence_interval_2'].astype(str) + "]"

    #Drop confidence interval columns
    df.drop(columns=['Confidence_interval_1', 'Confidence_interval_2'], inplace=True)
    df.drop(columns=['Confidence'], inplace=True)

    # Create a datafram for cycle time
    df_cycletime = df.loc[df['Measure'].str.contains('CT')].copy()
    df_cycletime.reset_index(drop=True, inplace=True)

    # Drop last row
    df_cycletime.drop(df_cycletime.tail(1).index, inplace=True)

    # Drop ct prefix
    df_cycletime['Measure'] = df_cycletime['Measure'].str.replace('CT_', '')

    # Rename the measures
    mapper = {
        'total': 'All',
        'FU': 'Follow-up',
        'R': 'Return',
        'N': 'New'
    }

    df_cycletime['Measure'] = df_cycletime['Measure'].replace(mapper)

    # Create a dataframe for queue
    df_queue = df.loc[df['Measure'].str.contains('Queue')].copy()
    df_queue.reset_index(drop=True, inplace=True)

    # Remove Queue prefix
    df_queue['Measure'] = df_queue['Measure'].str.replace('Queue_', '')

    # Rename the measures
    mapper = {
        'Working': 'Waiting for resources',
        'Waiting_Room': 'Waiting Room',
        'frontdesk': 'Front Desk',
        'total_c': 'Total',
        'FU': 'Follow-up',
        'R': 'Return',
        'N': 'New'
    }

    df_queue['Measure'] = df_queue['Measure'].replace(mapper)

    df_queue['Measure'] = df_queue['Measure'].str.replace('total_l', 'Total')
    # remove row 4 /index 3
    df_queue = df_queue.drop(df_queue.index[3])

    return df_cycletime, df_queue




In [4]:
def convert_to_latex(dataframe):
    # Function to convert a dataframe to a latex table
    return dataframe.to_latex(index=False, escape=False)

In [5]:
# The input files
file_name_dat = "Missing Residents.dat"
file_name_wip = "Missing Residents-WIP-logger.log"

df_cycletime, df_queue = get_CT_and_Queue(file_name_dat, 1) # Scenario 1
df_wip = get_WIP(file_name_wip,1)

print("Cycle time\n")
print(convert_to_latex(df_cycletime))

print("Queue\n")
print(convert_to_latex(df_queue))

print("WIP\n")
print(df_wip.to_latex())


Read the file
Cycle time

\begin{tabular}{lrrl}
\toprule
  Measure &    Mean &      SD & Confidence Interval \\
\midrule
      All & 215.624 & 135.941 &  [213.636, 217.612] \\
Follow-up &  68.888 &  26.851 &     [68.19, 69.585] \\
   Return & 293.300 & 108.813 &  [290.391, 296.208] \\
      New & 262.333 & 113.055 &   [259.27, 265.395] \\
\bottomrule
\end{tabular}

Queue

\begin{tabular}{lrrl}
\toprule
              Measure &    Mean &      SD & Confidence Interval \\
\midrule
Waiting for resources &  53.359 &  41.871 &    [53.039, 53.679] \\
          Wating_Room & 107.464 & 102.206 &  [105.821, 109.107] \\
           Front Desk &   5.061 &  11.424 &      [4.937, 5.185] \\
                Total & 165.884 & 129.990 &  [163.996, 167.772] \\
            Follow-up &  32.387 &  23.379 &    [31.846, 32.929] \\
               Return & 244.040 & 109.103 &   [241.241, 246.84] \\
                  New & 198.450 & 112.695 &   [195.57, 201.329] \\
\bottomrule
\end{tabular}

WIP

\begin{tabular}{l

In [6]:
# The input files
file_name_dat = "Missing Residents.dat"
file_name_wip = "Missing Residents-WIP-logger.log"

df_cycletime, df_queue = get_CT_and_Queue(file_name_dat, 2) # Scenario 2
df_wip = get_WIP(file_name_wip, 2)

print("Cycle time\n")
print(convert_to_latex(df_cycletime))

print("Queue\n")
print(convert_to_latex(df_queue))

print("WIP\n")
print(df_wip.to_latex())

Read the file
Cycle time

\begin{tabular}{lrrl}
\toprule
  Measure &    Mean &     SD & Confidence Interval \\
\midrule
      All & 102.583 & 40.276 &  [101.639, 103.526] \\
Follow-up &  73.339 & 27.881 &     [72.559, 74.12] \\
   Return & 112.096 & 36.961 &   [110.702, 113.49] \\
      New & 118.307 & 36.442 &  [117.109, 119.505] \\
\bottomrule
\end{tabular}

Queue

\begin{tabular}{lrrl}
\toprule
              Measure &   Mean &     SD & Confidence Interval \\
\midrule
Waiting for resources & 15.861 & 15.296 &    [15.612, 16.111] \\
          Wating_Room & 31.941 & 28.740 &     [31.33, 32.551] \\
           Front Desk &  4.966 & 11.268 &      [4.849, 5.083] \\
                Total & 52.768 & 33.623 &    [51.945, 53.591] \\
            Follow-up & 36.890 & 24.413 &    [36.284, 37.495] \\
               Return & 62.738 & 34.326 &    [61.481, 63.995] \\
                  New & 54.444 & 32.078 &    [53.435, 55.453] \\
\bottomrule
\end{tabular}

WIP

\begin{tabular}{lrrr}
\toprule
{} &  A